In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import wget
from datetime import datetime

In [2]:
url = "https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela1737.csv&terr=N&rank=-&query=t/1737/n1/all/v/all/p/all/d/v63%202,v69%202,v2263%202,v2264%202,v2265%202,v2266%2013/l/,v,p%2Bt"

ano_mes = datetime.today().date().replace(month=1).strftime('%b%y').lower()

wget.download(url, out=f'data\\ipca_{ano_mes}.csv')

'data\\ipca_jan22.csv'

In [3]:
ipca_file = f'data\\ipca_{ano_mes}.csv'

df = pd.read_csv(ipca_file, sep=',', encoding='utf-8-sig', skiprows=2, low_memory=False)

# Renomeando colunas
cols_name = {
    'Unnamed: 1': 'Território',
    'IPCA - Número-índice (base: dezembro de 1993 = 100) (Número-índice)': 'Número-índice',
    'IPCA - Variação mensal (%)': 'Variação mensal (%)', 'IPCA - Variação acumulada em 3 meses (%)': 'Variação 3 meses (%)',
    'IPCA - Variação acumulada em 6 meses (%)': 'Variação 6 meses (%)', 'IPCA - Variação acumulada no ano (%)': 'Variação anual (%)',
    'IPCA - Variação acumulada em 12 meses (%)': 'Variação 12 meses (%)'
    }

df.rename(columns=cols_name, inplace=True)

# Substituindo pontos no DataFrame
df = df.replace('...', 0)

# Retirando as linhas como fontes e notas
df = df.iloc[:-12]

mes = {'janeiro': 1, 'fevereiro': 2, 'março': 3, 'abril': 4, 'maio': 5, 'junho': 6, 'julho': 7,
        'agosto': 8, 'setembro': 9, 'outubro': 10, 'novembro': 11, 'dezembro': 12}

# Dividindo a coluna de Mês para obter informação do Ano
df[['Mês', 'Ano']] = df['Mês'].str.split(' ', expand=True)

df['Mes_num'] = ''

# Preenchendo a coluna Mes_num de acordo com as chaves da variavel mes
df['Mes_num'] = df['Mês'].map(mes)

# Escolhendo colunas a serem visualizadas
df = df[['Ano', 'Mes_num', 'Mês', 'Número-índice', 'Variação mensal (%)',
    'Variação 3 meses (%)', 'Variação 6 meses (%)',
    'Variação anual (%)', 'Variação 12 meses (%)']]

df


,Ano,Mes_num,Mês,Número-índice,Variação mensal (%),Variação 3 meses (%),Variação 6 meses (%),Variação anual (%),Variação 12 meses (%)
0,1979,12,dezembro,7.618300e-09,0,0,0,0,0
1,1980,1,janeiro,8.122300e-09,6.62,0,0,6.62,0
2,1980,2,fevereiro,8.497300e-09,4.62,0,0,11.54,0
3,1980,3,março,9.010400e-09,6.04,18.27,0,18.27,0
4,1980,4,abril,9.486700e-09,5.29,16.80,0,24.53,0
...,...,...,...,...,...,...,...,...,...
501,2021,9,setembro,5.944210e+03,1.16,3.02,4.75,6.90,10.25
502,2021,10,outubro,6.018510e+03,1.25,3.32,5.73,8.24,10.67
503,2021,11,novembro,6.075690e+03,0.95,3.40,5.86,9.26,10.74
504,2021,12,dezembro,6.120040e+03,0.73,2.96,6.07,10.06,10.06


In [4]:
df['Ano'] = df['Ano'].astype(int)

df[['Número-índice', 'Variação mensal (%)', 'Variação 3 meses (%)', 'Variação 6 meses (%)',
    'Variação anual (%)', 'Variação 12 meses (%)']] = df[['Número-índice', 'Variação mensal (%)',
    'Variação 3 meses (%)', 'Variação 6 meses (%)',
    'Variação anual (%)', 'Variação 12 meses (%)']].astype(float)

In [5]:
df.to_csv(f'output\\ipca_formatado_{ano_mes}.csv', index=False, decimal=',', encoding='utf-8-sig', sep=';')

## Gráficos

In [6]:
# Criando uma cópia do DataFrame original
df_2 = df.copy()
df_2.head(4)

,Ano,Mes_num,Mês,Número-índice,Variação mensal (%),Variação 3 meses (%),Variação 6 meses (%),Variação anual (%),Variação 12 meses (%)
0,1979,12,dezembro,7.618300e-09,0.00,0.00,0.0,0.00,0.0
1,1980,1,janeiro,8.122300e-09,6.62,0.00,0.0,6.62,0.0
2,1980,2,fevereiro,8.497300e-09,4.62,0.00,0.0,11.54,0.0
3,1980,3,março,9.010400e-09,6.04,18.27,0.0,18.27,0.0


In [8]:
# Realizando um filtro para selecionar informações posteriores a 1999 para apenas o mês de Dezembro
df_2 = df_2[(df_2['Ano'] > 1999) & (df_2['Mês'].str.contains('dezembro'))]
df_2.head(4)

,Ano,Mes_num,Mês,Número-índice,Variação mensal (%),Variação 3 meses (%),Variação 6 meses (%),Variação anual (%),Variação 12 meses (%)
252,2000,12,dezembro,1683.47,0.59,1.05,4.26,5.97,5.97
264,2001,12,dezembro,1812.65,0.65,2.21,4.58,7.67,7.67
276,2002,12,dezembro,2039.78,2.10,6.56,9.31,12.53,12.53
288,2003,12,dezembro,2229.49,0.52,1.15,2.49,9.30,9.30


In [9]:
# Utilizando algumas features do Plotly para criar um gráfico

# Passando o DataFrame como fonte de dados e escolhendo os eixos devidos
fig = px.bar(df_2, x = 'Ano', y = 'Variação anual (%)', text_auto = '.1f')

# Definindo os rótulos de dados como fonte 10 e alocados fora das colunas internas
fig.update_traces(textfont_size = 7, textangle = 0, textposition = 'outside', marker_color='midnightblue')

# Ajustando as informações do gráfico, adicionando título, centralizando e formatando eixos
fig.update_layout(
    title = {
        'text':'Inflação nos últimos anos <br><sup>Variação anual, em %',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size':20, 'color': 'darkslategrey', 'family': 'luminari, fantasy'}
    },
    yaxis = dict(
        title = '(%)',
        titlefont_size=12),
    xaxis = dict(
        title = 'Período',
        titlefont_size = 12,
        type = 'category'),
    uniformtext_minsize = 10,
    uniformtext_mode = 'hide',
    paper_bgcolor='rgb(255,255,255)',
    plot_bgcolor='rgb(255,255,255)',
)

# Ajustando o ângulo do eixo X
fig.update_xaxes(tickangle=-60)

fig.show()

In [10]:
fig.write_image('output\\figura_final.jpeg', width=1280, height=720)